<a href="https://colab.research.google.com/github/souma21122002/Internship2025/blob/main/Celeb_df_frame_face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install mediapipe==0.10.20 --force-reinstall

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.5/104.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 97.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3

In [ ]:
!pip install mediapipe==0.10.20
!pip install opencv-python


In [ ]:

import zipfile
zip_ref=zipfile.ZipFile('/content/drive/MyDrive/celeb_DF.zip')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:


import cv2
import mediapipe as mp
import os
from google.colab.patches import cv2_imshow

# Define paths
input_folder = '/content/Celeb-real'
cropped_faces_folder = '/content/Celeb_real_Mediapipe/faces_mediapipe/'
original_frames_folder = '/content/Celeb_real_Mediapipe/frames_mediapipe/'

# Create output directories if they don't exist
os.makedirs(cropped_faces_folder, exist_ok=True)
os.makedirs(original_frames_folder, exist_ok=True)

# Initialize MediaPipe Face Detection
mp_face_detection = mp.solutions.face_detection
DETECTION_CONFIDENCE = 0.5

with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=DETECTION_CONFIDENCE) as face_detector:
    for video_file in os.listdir(input_folder):
        video_path = os.path.join(input_folder, video_file)

        if video_path.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
            print(f"Processing video: {video_file}")
            base_name = os.path.splitext(video_file)[0]

            # Create subfolders for each video
            video_cropped_folder = os.path.join(cropped_faces_folder, base_name)
            video_real_frame_folder = os.path.join(original_frames_folder, base_name)
            os.makedirs(video_cropped_folder, exist_ok=True)
            os.makedirs(video_real_frame_folder, exist_ok=True)

            cap = cv2.VideoCapture(video_path)
            frame_count = 0

            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                frame_count += 1

                # Convert BGR to RGB for MediaPipe
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = face_detector.process(rgb_frame)

                if results.detections:
                    # Save the full frame (original) if face is detected
                    real_frame_filename = f"{base_name}_frame_{frame_count:06d}.jpg"
                    real_frame_path = os.path.join(video_real_frame_folder, real_frame_filename)
                    cv2.imwrite(real_frame_path, frame)

                    # Save cropped faces
                    for i, detection in enumerate(results.detections):
                        bboxC = detection.location_data.relative_bounding_box
                        ih, iw, _ = frame.shape
                        x1 = int(bboxC.xmin * iw)
                        y1 = int(bboxC.ymin * ih)
                        w = int(bboxC.width * iw)
                        h = int(bboxC.height * ih)
                        x2 = x1 + w
                        y2 = y1 + h

                        # Ensure coordinates are within frame bounds
                        cropped_face = frame[max(0, y1):min(ih, y2), max(0, x1):min(iw, x2)]

                        if cropped_face.shape[0] > 0 and cropped_face.shape[1] > 0:
                            cropped_filename = f"{base_name}_face_{frame_count:06d}.jpg"
                            cropped_path = os.path.join(video_cropped_folder, cropped_filename)
                            cv2.imwrite(cropped_path, cropped_face)

            cap.release()
            print(f"Finished processing: {video_file}")
            print(f"→ Real frames saved to: {video_real_frame_folder}")
            print(f"→ Cropped faces saved to: {video_cropped_folder}")

cv2.destroyAllWindows()

/usr/local/lib/python3.11/dist-packages/jaxlib/plugin_support.py:71: RuntimeWarning: JAX plugin jax_cuda12_plugin version 0.5.1 is installed, but it is not compatible with the installed jaxlib version 0.6.1, so it will not be used.
  warnings.warn(


Processing video: id44_0001.mp4
Finished processing: id44_0001.mp4
→ Real frames saved to: /content/Celeb_real_Mediapipe/frames_mediapipe/id44_0001
→ Cropped faces saved to: /content/Celeb_real_Mediapipe/faces_mediapipe/id44_0001
Processing video: id33_0000.mp4
Finished processing: id33_0000.mp4
→ Real frames saved to: /content/Celeb_real_Mediapipe/frames_mediapipe/id33_0000
→ Cropped faces saved to: /content/Celeb_real_Mediapipe/faces_mediapipe/id33_0000
Processing video: id51_0004.mp4
Finished processing: id51_0004.mp4
→ Real frames saved to: /content/Celeb_real_Mediapipe/frames_mediapipe/id51_0004
→ Cropped faces saved to: /content/Celeb_real_Mediapipe/faces_mediapipe/id51_0004
Processing video: id30_0009.mp4
Finished processing: id30_0009.mp4
→ Real frames saved to: /content/Celeb_real_Mediapipe/frames_mediapipe/id30_0009
→ Cropped faces saved to: /content/Celeb_real_Mediapipe/faces_mediapipe/id30_0009
Processing video: id27_0008.mp4
Finished processing: id27_0008.mp4
→ Real frames

In [ ]:
import os

# Set your directories
base_dir = "/content/Celeb_real_Mediapipe"
faces_dir = os.path.join(base_dir, "faces_mediapipe")
frames_dir = os.path.join(base_dir, "frames_mediapipe")

# Get all files
face_files = []
frame_files = []

for root, _, files in os.walk(faces_dir):
    face_files.extend(f for f in files if '_face_' in f)

for root, _, files in os.walk(frames_dir):
    frame_files.extend(f for f in files if '_frame_' in f)

# Create mapping dictionaries
face_dict = {f.replace('_face_', '_'): f for f in face_files}
frame_dict = {f.replace('_frame_', '_'): f for f in frame_files}

# Calculate matches and mismatches
connected_pairs = set(face_dict.keys()) & set(frame_dict.keys())
faces_without_frames = set(face_dict.keys()) - set(frame_dict.keys())
frames_without_faces = set(frame_dict.keys()) - set(face_dict.keys())

# Print results
print("=== Summary ===")
print(f"Total face images: {len(face_files)}")
print(f"Total frame images: {len(frame_files)}")
print(f"Connected pairs: {len(connected_pairs)}")
print(f"Faces without frames: {len(faces_without_frames)}")
print(f"Frames without faces: {len(frames_without_faces)}")

# Print sample mismatches (first 5 of each)
print("\n=== Sample Faces Without Frames ===")
for i, key in enumerate(list(faces_without_frames)[:5]):
    print(f"{i+1}. {face_dict[key]}")

print("\n=== Sample Frames Without Faces ===")
for i, key in enumerate(list(frames_without_faces)[:5]):
    print(f"{i+1}. {frame_dict[key]}")

=== Summary ===
Total face images: 222305
Total frame images: 222305
Connected pairs: 222305
Faces without frames: 0
Frames without faces: 0

=== Sample Faces Without Frames ===

=== Sample Frames Without Faces ===


In [ ]:
from google.colab import files
import shutil
import os

dir_to_download = "/content/Celeb_real_Mediapipe"

output_zip = "Celeb_real_Mediapipe_Output.zip"
shutil.make_archive(output_zip[:-4], 'zip', dir_to_download)


files.download(output_zip)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
from google.colab import files
import shutil
import os

files.download("/content/Celeb_real_Mediapipe_Output.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>